# EDA 

In [10]:
import numpy as np      
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
os.chdir("bbc")
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:

with open("data_for_embeddings.txt") as f:
    tempf = f.readlines()   
tempf = ' '.join(tempf)                  #tempf is a string holding the text from data_for_embedding_file

In [15]:
# https://www.udemy.com/course/machinelearning/
def cleaner(str):
    review = re.sub('[^a-zA-Z]', ' ',str )   #uppercase to lowercase
    review = review.lower()                 
    review = review.split()                  #splitting so that stemming can be done
    ps = PorterStemmer()                     
                                             #remove stopwords and stem remaining ones 
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)                
    str=review
    return review

In [16]:
tempf=cleaner(tempf)
print(tempf)

dollar gain greenspan speech dollar hit highest level euro almost three month feder reserv head said us trade deficit set stabilis alan greenspan highlight us govern willing curb spend rise household save factor may help reduc late trade new york dollar reach euro thursday market concern deficit hit greenback recent month friday feder reserv chairman mr greenspan speech london ahead meet g financ minist sent dollar higher earlier tumbl back wors expect us job data think chairman take much sanguin view current account deficit taken time said robert sinch head currenc strategi bank america new york take longer term view lay set condit current account deficit improv year next worri deficit concern china howev remain china currenc remain peg dollar us currenc sharp fall recent month therefor made chines export price highli competit call shift beij polici fallen deaf ear despit recent comment major chines newspap time ripe loosen peg g meet thought unlik produc meaning movement chines polic

In [13]:
# now opening classify_text in tempf as string
with open("classify_text.txt") as f:
    tempf = f.readlines()
tempf = ' '.join(tempf)
tempf=cleaner(tempf)

In [14]:
print(tempf)

white collar compani race last return offic googl facebook amazon capit one other extend work home polici septemb sometim far beyond credit jackson gibb david streitfeld david streitfeld may even presid trump said get countri open much corpor america rush return employe campus skyscrap compani race first back last increas number mostli white collar employe recent extend work home polici far beyond shelter place timelin mandat state local author googl facebook employe told thursday could stay home next year capit one inform worker labor day possibl longer amazon say octob nationwid insur move aggress firm shutter five offic around countri employe telecommut perman move reflect realiti one sure coronaviru pandem evolv death viru hot zone like new york citi come new outbreak emerg elsewher almost everi day least new case u bring countri total million even coronaviru longer requir work home like retain signific presenc corpor life affect shape citi commerci real estat industri chang cultur

In [15]:
#Make a folder bbc whuch contains all five sub folders and contains embedding_for_text and classify_text
# data_folder = "bbc"
folders = ["business","entertainment","politics","sport","tech"]

# os.chdir(data_folder)

x = []
y = []

for i in folders:
    files = os.listdir(i)
    for text_file in files:
        file_path = i + "/" +text_file
        with open(file_path) as f:
            data = f.readlines()
        data = ' '.join(data)
        x.append(data)
        y.append(i)
   
data = {'news': x, 'type': y}     #Headers  
df = pd.DataFrame(data)
df.to_csv('dataset.csv', index=False)      

#Importing the dataset
dataset = pd.read_csv('dataset.csv')

In [36]:
#2225
corpus = []
for i in range(0, 2225):
    review = re.sub('[^a-zA-Z]', ' ', dataset['news'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
corpus.append(tempf)             #tempf file i.e, classify_text is append to corpus to ease

# Bag of Words model

In [82]:
# https://www.udemy.com/course/machinelearning/
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
print(len(X))

2226


In [83]:
y = dataset.iloc[:, 1].values
from sklearn import preprocessing
le = preprocessing.LabelEncoder()     # y labels were strings so label so encoding was necessary 
le.fit_transform(y)                   
len(y)

2225

In [84]:
# https://www.udemy.com/course/machinelearning/
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X[:-1], y, test_size = 0.10, random_state = 0)

# Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[49  0  3  0  5]
 [ 0 30  1  0  2]
 [ 4  1 42  0  2]
 [ 1  2  0 47  0]
 [ 2  1  0  0 31]]


In [90]:
type(X[2225])

numpy.ndarray

In [92]:
ypre=classifier.predict((X[2225]).reshape(1,-1))
print(ypre)

['tech']


# TFIDF MODEL 

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
Xtfid=vectorizer.fit_transform(corpus).toarray()

In [99]:
len(Xtfid)

2226

In [93]:
X_train, X_test, y_train, y_test = train_test_split(Xtfid[:-1], y, test_size = 0.20, random_state = 0,stratify=y)

# Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[85  6  6  0  5]
 [ 2 72  0  0  3]
 [ 5  1 76  1  1]
 [ 0  2  1 99  0]
 [ 1  1  2  0 76]]


In [100]:
y_pre=classifier.predict((Xtfid[2225]).reshape(1,-1))
print(y_pre)

['tech']


In [ ]:
Thus all models predict tech as the data_for_prediction